## Imports

In [392]:

import numpy as np
import math
from fractions import Fraction
import itertools

## Constants

In [407]:
#CONSTANT

pyt_comma = 0.0136432
peaks = [2.4, 8.7, 11.4, 18.8, 29, 33.6]
round_peaks = [int(i) for i in peaks]

#planet ratios
Schumann_earth = [7.83, 14.3, 20.8, 27.3, 33.8]
Mercury = 141.27

## Functions

### Utilities

In [265]:
def rebound(x, low = 1, high = 2, octave = 2):
    while x > high:
        x = x/octave
    while x < low:
        x = x*octave
    return x

def nth_root (num, root):
    answer = num**(1/root)
    return answer


#Function that compares lists (i.e. peak harmonics)
def compareLists(list1, list2, bounds):
    matching = []
    matching_pos = []
    for i, l1 in enumerate(list1):
        for j, l2 in enumerate(list2):
            if l2-bounds < l1 < l2+bounds:
                matching.append((l1+l2)/2)
                matching_pos.append([(l1+l2)/2, i+1, j+1])
    matching = np.array(matching)
    matching_pos = np.array(matching_pos)
    ratios_temp = []
    for i in range(len(matching_pos)):
        if matching_pos[i][1]>matching_pos[i][2]:
            ratios_temp.append(matching_pos[i][1]/matching_pos[i][2])
        else:
            ratios_temp.append(matching_pos[i][2]/matching_pos[i][1])
    matching_pos_ratios = np.array(ratios_temp)
    return matching, matching_pos, matching_pos_ratios

In [343]:
a, b, c = compareLists([2.4, 4.8, 5.8], [5.8, 4, 8, 10], 1)
a

array([4.4, 5.8])

### Peak ratios and Harmonics

In [71]:
#This function calculates the 15 ratios (with the possibility to bound them between 1 and 2) derived from the 6 peaks
def compute_peak_ratios(peaks, rebound = 1, octave = 2):
    ratios = []
    peak_ratios_rebound = []
    for p1 in peaks:
        for p2 in peaks:
            ratio_temp = p2/p1
            if ratio_temp == 1:
                ratio_temp = None
            elif ratio_temp < 1:
                ratio_temp = None

            ratios.append(ratio_temp)

        peak_ratios = np.array(ratios)
        peak_ratios = [i for i in peak_ratios if i]
        peak_ratios = list(set(peak_ratios))
    if rebound == 1:
        for peak in peak_ratios:
            while peak > octave:
                peak = peak/octave
            peak_ratios_rebound.append(peak)
    peak_ratios_rebound = np.array(peak_ratios_rebound)
    peak_ratios_rebound = [i for i in peak_ratios_rebound if i]
    peak_ratios = sorted(peak_ratios)
    peak_ratios_rebound = sorted(peak_ratios_rebound)
    return peak_ratios, peak_ratios_rebound

In [420]:
#This function takes a list of frequency peaks as input and computes the desired number of harmonics
#with the formula: x + 2x + 3x ... + nx
def EEG_harmonics_mult(peaks, n_harmonics, n_oct_up = 0, rebound = 1):
    n_harmonics = n_harmonics + 2
    multi_harmonics = []
    multi_harmonics_rebound = []
    for p in peaks:
        multi_harmonics_r = []
        multi_harm_temp = []
        harmonics = []
        p = p * (2**n_oct_up)
        i = 1
        harm_temp = p
        while i < n_harmonics:
            harm_temp = p * i
            harmonics.append(harm_temp)
            i+=1
        multi_harmonics.append(harmonics)
    multi_harmonics = np.array(multi_harmonics)  
        
    return multi_harmonics

#This function takes a list of frequency peaks as input and computes the desired number of harmonics
#with the formula: x + x/2 + x/3 ... + x/n
def EEG_harmonics_div(peaks, n_harmonics, n_oct_up = 0):
    n_harmonics = n_harmonics + 2
    multi_harmonics = []
    multi_harmonics_sub = []
    for p in peaks:
                           
        harmonics = [] 
        harmonics_sub = []
        p = p * (2**n_oct_up)               
        i = 2
        harm_temp = p
        harm_temp_sub = p
        while i < n_harmonics:
            harm_temp = harm_temp + (p/i)
            harm_temp_sub = abs(harm_temp_sub - (p/i))
            harmonics.append(harm_temp)
            harmonics_sub.append(harm_temp_sub)
            i+=1    
        multi_harmonics.append(harmonics)
        multi_harmonics_sub.append(harmonics_sub)
    multi_harmonics = np.array(multi_harmonics)
    multi_harmonics_bounded = multi_harmonics.copy()
    multi_harmonics_sub = np.array(multi_harmonics_sub)
    multi_harmonics_sub_bounded = multi_harmonics_sub.copy()
    #Rebound the result between 1 and 2
    for i in range(len(multi_harmonics_bounded)):
        for j in range(len(multi_harmonics_bounded[0])):
            multi_harmonics_bounded[i][j] = rebound(multi_harmonics_bounded[i][j])
            multi_harmonics_sub_bounded[i][j] = rebound(multi_harmonics_sub_bounded[i][j])
    return multi_harmonics, multi_harmonics_bounded, multi_harmonics_sub, multi_harmonics_sub_bounded


def harmonic_fit(peak_bands = [0, 1, 2, 3, 4, 5], n_harm = 10, bounds = 1, peaks = peaks):
    from itertools import combinations

    multi_harmonics = EEG_harmonics_mult(peaks, 50, 0)
    #print(multi_harmonics)
    list_peaks = list(combinations(peak_bands,2))
    #print(list_peaks)
    harm_temp = []
    for i in range(len(list_peaks)):
        harms, b, c = compareLists(multi_harmonics[list_peaks[i][0]], multi_harmonics[list_peaks[i][1]], bounds) 
        
        harm_temp.append(harms)
        
    harm_fit = np.array(harm_temp).squeeze()
    
    #print(harm_fit)
    if len(peak_bands) > 2:
        harm_fit = list(itertools.chain.from_iterable(harm_fit))

    return harm_fit

In [120]:
mh, mh_bound, mh_sub, mh_sub_bound = EEG_harmonics_div(peaks, 5, n_oct_up = 0)
mh_bound

array([[1.8       , 1.1       , 1.25      , 1.37      , 1.47      ],
       [1.63125   , 1.99375   , 1.1328125 , 1.2415625 , 1.3321875 ],
       [1.06875   , 1.30625   , 1.484375  , 1.626875  , 1.745625  ],
       [1.7625    , 1.07708333, 1.22395833, 1.34145833, 1.439375  ],
       [1.359375  , 1.66145833, 1.88802083, 1.03463542, 1.11015625],
       [1.575     , 1.925     , 1.09375   , 1.19875   , 1.28625   ]])

### n-TET 

In [73]:


def oct_subdiv(ratio,bounds,octave,n):    
    Octdiv, Octvalue, i = [], [], 1
    ratios = []
    while len(Octdiv) < n:
        ratio_mult = (ratio**i)
        while ratio_mult > octave:
            ratio_mult = ratio_mult/octave    
            
        rescale_ratio = ratio_mult - round(ratio_mult)
        ratios.append(ratio_mult)
        i+=1
        if -bounds < rescale_ratio < bounds:
            Octdiv.append(i-1)
            Octvalue.append(ratio_mult)
        else:
            continue
    return Octdiv, Octvalue, ratios


def compare_octave_division(Octdiv = 53, Octdiv2 = 12, bounds = 0.01, octave = 2):
    ListOctdiv = []
    ListOctdiv2 = []
    OctdivSum = 1
    OctdivSum2 = 1
    i = 1
    i2 = 1
    Matching_harmonics = []
    #HARMONIC_RATIOS = [1, 1.0595, 1.1225, 1.1892, 1.2599, 1.3348, 1.4142, 1.4983, 1.5874, 1.6818, 1.7818, 1.8897]
    while OctdivSum < octave:
        OctdivSum =(nth_root(octave, Octdiv))**i
        i+=1
        ListOctdiv.append(OctdivSum)
    #print(ListOctdiv)

    while OctdivSum2 < octave:
        OctdivSum2 =(nth_root(octave, Octdiv2))**i2
        i2+=1
        ListOctdiv2.append(OctdivSum2)
    #print(ListOctdiv2)
    for i, n in enumerate(ListOctdiv):
        for j, harm in enumerate(ListOctdiv2):
            if harm-bounds < n < harm+bounds:
                Matching_harmonics.append([n, i+1, harm, j+1])
    Matching_harmonics = np.array(Matching_harmonics)
    return Matching_harmonics




### Consonance

In [74]:
#Oct_subdiv
#Argument 1 : a ratio in the form of a float or a fraction
#Argument 2 : bounds between which the octave should fall
#Argument 3 : value of the octave
#Argument 4 : number of octave subdivisions
def compute_consonance (peaks, limit):
    consonance_ = []
    peaks2keep = []
    peaks_consonance = []
    for p1 in peaks:    
        for p2 in peaks:
            peaks2keep_temp = []
            p2x = p2
            p1x = p1
            cons_temp = (p2/p1).as_integer_ratio()
            cons_temp = (cons_temp[0] + cons_temp[1])/(cons_temp[0] * cons_temp[1])
            #print(cons_temp)
            if cons_temp == 2:
                cons_temp = None
                p2x = None
                p1x = None
            elif cons_temp < limit:
                cons_temp = None
                p2x = None
                p1x = None
            if p2x != None:
                peaks2keep_temp.extend([p2x, p1x])
            consonance_.append(cons_temp)  
            peaks2keep.append(peaks2keep_temp)
        peaks_consonance = np.array(peaks2keep)
        peaks_consonance = [x for x in peaks_consonance if x]
        consonance = np.array(consonance_)
        consonance = [i for i in consonance if i]
        #consonance = list(set(consonance))
    return consonance, peaks_consonance

# Function that computes integer ratios from peaks with higher consonance
def comp_consonant_integers_ratios (peaks_consonance):
    cons_integer = []
    for i in range(len(peaks_consonance)):
        #for j in range(len(peaks_consonance[0])):
        a = peaks_consonance[i][0]
        b = peaks_consonance[i][1]
        if a > b:
            while a > (b*2):
                a = a/2
        elif a < b:
            while b > (a*2):
                b = b/2
        cons_temp = (a/b).as_integer_ratio()
        cons_integer.append(cons_temp)
    consonant_integers = np.array(cons_integer).squeeze()
    cons_ratios = []
    for j in range(len(consonant_integers)):
        cons_ratios.append((consonant_integers[j][0])/(consonant_integers[j][1]))
    consonant_ratios = np.array(cons_ratios)
    return consonant_integers, consonant_ratios

In [228]:
a, b = compute_consonance(peaks + Schumann_earth, 0.05)
c, d = comp_consonant_integers_ratios (b)
d

array([1.8125  , 1.1875  , 1.421875, 0.8125  , 0.6875  , 1.3125  ])

In [171]:
2*53/12

8.833333333333334

### Tunings

In [186]:
# Function that computes Pythogorian Tunings from integer ratios 
# List of equations of the Pythagorean Tuning
# variable 'a' has to be the highest value in the ratio

#INPUTS (a: numerator of the source ratio / b: denominator of the source ratio
#        n: multiplifier of all notes / octa: octave value / lim_denom: highest value possible
#        for the denominator of the ratios)
#OUTPUTS

def compute_pythagore(a = 3, b = 2, n = 1, octa = 2, lim_denom = 1000):
    import operator
    pyth0 = n * (1/1)
    pyth1 = n * ((b/a)**5)*(octa**3)
    pyth1 = rebound(pyth1)
    pyth2 = n * ((a/b)**2)*(1/octa)
    pyth2 = rebound(pyth2)
    pyth3 = n * ((b/a)**3)*(octa**2)
    pyth3 = rebound(pyth3)
    pyth4 = n * ((a/b)**4)*((1/octa)**2)
    pyth4 = rebound(pyth4)
    pyth5 = n * (b/a)*octa
    pyth5 = rebound(pyth5)
    pyth6 = n * ((b/a)**6)*(octa**4)
    pyth6 = rebound(pyth6)
    pyth6_2 = n * ((a/b)**6)*((1/octa)**3)
    pyth6_2 = rebound(pyth6_2)
    pyth7 = n * (a/b)
    pyth7 = rebound(pyth7)
    pyth8 = n * ((b/a)**4)*(octa**3)
    pyth8 = rebound(pyth8)
    pyth9 = n * ((a/b)**3)*(1/octa)
    pyth9 = rebound(pyth9)
    pyth10 = n * ((b/a)**2)*(octa**2)
    pyth10 = rebound(pyth10)
    pyth11 = n * ((a/b)**5)*((1/octa)**2)
    pyth11 = rebound(pyth11)
    pyth12 = rebound(n * octa)
    
    pyth_dict = {'unison':pyth0,
             'minor second':pyth1,
             'major second':pyth2,
             'minor third':pyth3,
             'major third':pyth4,
             'perfect fourth':pyth5,
             'diminished fifth':pyth6,
             'augmented fourth':pyth6_2,
             'perfect fifth':pyth7,
             'minor sixth':pyth8,
             'major sixth':pyth9,
             'minor seventh':pyth10,
             'major seventh':pyth11,
             'octave':pyth12}
    pyth_dict_list = sorted(pyth_dict.items(), key=operator.itemgetter(1))
    pyth_dict = dict(pyth_dict)
    pyth_temp = []
    pyth_temp_float = []
    for i in range(len(pyth_dict_list)):
        frac = []
        #print(harm_tuning[i])
        pyth_ratios = Fraction(pyth_dict_list[i][1]).limit_denominator(lim_denom)
        pyth_float = pyth_dict_list[i][1]
        num = pyth_ratios.numerator
        denom = pyth_ratios.denominator
        frac.extend([num, denom])
        #print(harm_ratios_temp)
        pyth_temp.append(frac)
        pyth_temp_float.append(pyth_float)
       
    pyth_dict_list_ratios = (np.array(pyth_temp)).squeeze()
    pyth_dict_list_float = np.array(pyth_temp_float)
    
    return pyth_dict, pyth_dict_list, pyth_dict_list_ratios, pyth_dict_list_float

#This Function build a tuning based on the multiplication of a ratio by itself.
def harmonic_tuning(n_harmonics = 10, octave = 2, rounding = 10, pos = 1):
    harm_tuning_temp = []
    i = 1
    harm_ratios = []
    harm_ratios_temp = []
    while i <= n_harmonics:
        harm_temp = pos*i
        harm_tuning_temp.append(harm_temp)
        i+=1
    
    for h in harm_tuning_temp:
                while h > pos*octave:
                    h = h/octave
                while h > octave:
                    h = h/pos
                h = round(h, rounding)

                #if h == octave:
                #    h = None
                harm_ratios_temp.append(h)
    harm_tuning = np.array(harm_ratios_temp)
    harm_tuning = [i for i in harm_tuning if i]
    harm_tuning = list(set(harm_tuning))
    for i in range(len(harm_tuning)):
        frac = []
        #print(harm_tuning[i])
        harm_ratios_temp = Fraction(harm_tuning[i]).limit_denominator()
        num = harm_ratios_temp.numerator
        denom = harm_ratios_temp.denominator
        frac.extend([num, denom])
        #print(harm_ratios_temp)
        harm_ratios.append(frac)
    
    harm_tuning_frac = (np.array(harm_ratios)).squeeze().tolist()

    return harm_tuning, harm_tuning_frac

#This function subdivides the octave in equal parts
def Ntet_tuning(subdiv, octave):
    ratios_temp = []
    for i in range(subdiv+1):
        ratios_temp.append(octave**(i/subdiv))
    ratios = np.array(ratios_temp)
    return ratios

In [274]:
peak_bands = [0, 1, 2, 3, 4, 5]
len(peak_bands)

6

6

In [480]:

harm_fit = harmonic_fit([0, 1], 5, 1)

In [345]:
x = compareLists(multi_harmonics[list_peaks[0][1]], multi_harmonics[list_peaks[0][0]], 0.1)
x
#multi_harmonics[list_peaks[0][1]]

(array([69.6]), array([[69.6,  8. , 29. ]]), array([3.625]))

## Bidouille

In [190]:
#compute_consonance takes a list of peaks as input(1) and a threshold(2)
#above which the peaks are kept
#output(1) returns the consonance values
#output(2) return the peaks corresponding to the consonance values
consonant_values, consonant_peaks = compute_consonance(Schumann_earth+peaks, 0.01)
print('CONSONANT PEAKS')
print(consonant_peaks)
#This function takes a list of consonant peaks and returns the peak ratios
#with integer values
cons_integers = comp_consonant_integers_ratios(consonant_peaks)
print('CONSONANT INTEGER RATIOS')
print(cons_integers)

CONSONANT PEAKS
[[14.3, 20.8], [27.3, 20.8], [27.3, 2.4], [8.7, 2.4], [11.4, 2.4], [27.3, 33.6]]
CONSONANT INTEGER RATIOS
(array([[11, 16],
       [21, 16],
       [91, 64],
       [29, 16],
       [19, 16],
       [13, 16]]), array([0.6875  , 1.3125  , 1.421875, 1.8125  , 1.1875  , 0.8125  ]))


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [203]:

#Oct_subdiv
#Argument 1 : a ratio in the form of a float or a fraction
#Argument 2 : bounds between which the octave should fall
#Argument 3 : value of the octave
#Argument 4 : number of octave subdivisions
Octdiv, Octvalue, ratios_tet = oct_subdiv(7/5, 0.05, 2, 5)
print('OCTAVE SUBDIVISIONS')
print(Octdiv, Octvalue)
print('RATIOS-TET')
print(ratios_tet)
pairwise_oct_div = compare_octave_division(Octdiv = 23, Octdiv2 = 33, bounds = 0.005, octave = 2)
print('COMPARE OCTAVE SUBDIVISIONS')
print(pairwise_oct_div)




peak_ratios, peak_ratios_rebound = compute_peak_ratios(peaks, 1)
print('PEAK RATIOS')
print(peak_ratios, peak_ratios_rebound)



#Calculates a specific number of partials from a list of peaks [n_peaks][n_harmonic]
multi_harmonics= EEG_harmonics_mult(peaks, 50, 0)
print('HARMONICS (x*n)')
print(multi_harmonics)
#Calculates a specific number of elements in the harmonics series
#from a list of peaks [n_peaks][n_harmonic]
harm_div, harm_div_bound, harm_div_sub, harm_div_sub_bound = EEG_harmonics_div(peaks, n_harmonics = 8)
print('HARMONICS (x/n)')
print(harm_div_bound)

#pyth_dict, pyth_dict_list = compute_pythagore(cons_integers[2][0], cons_integers[2][1], 1, 2)
a, b, c, d = compute_pythagore(4, 3, 1, 2, 1000)
a

OCTAVE SUBDIVISIONS
[2, 31, 33, 35, 64] [1.9599999999999997, 1.0339967437035635, 1.0133168088294922, 1.986100945305804, 1.0477662806697812]
RATIOS-TET
[1.4, 1.9599999999999997, 1.3719999999999997, 1.9207999999999996, 1.3445599999999995, 1.8823839999999994, 1.3176687999999994, 1.844736319999999, 1.2913154239999993, 1.8078415935999987, 1.2654891155199992, 1.7716847617279987, 1.240179333209599, 1.7362510664934385, 1.215375746545407, 1.7015260451635694, 1.1910682316144985, 1.6674955242602978, 1.1672468669822085, 1.6341456137750916, 1.143901929642564, 1.6014627014995897, 1.1210238910497128, 1.5694334474695977, 1.0986034132287184, 1.5380447785202056, 1.0766313449641438, 1.5072838829498012, 1.0550987180648608, 1.477138205290805, 1.0339967437035635, 1.4475954411849887, 1.0133168088294922, 1.4186435323612887, 1.986100945305804, 1.3902706617140628, 1.9463789263996878, 1.3624652484797815, 1.907451347871694, 1.3352159435101856, 1.8693023209142596, 1.3085116246399817, 1.8319162744959743, 1.28234139

{'unison': 1.0,
 'minor second': 1.8984375,
 'major second': 0.8888888888888888,
 'minor third': 1.6875,
 'major third': 0.7901234567901233,
 'perfect fourth': 1.5,
 'diminished fifth': 2.84765625,
 'augmented fourth': 0.7023319615912206,
 'perfect fifth': 1.3333333333333333,
 'minor sixth': 2.53125,
 'major sixth': 1.185185185185185,
 'minor seventh': 2.25,
 'major seventh': 1.053497942386831,
 'octave': 2}

In [ ]:
a, b, c, d = compute_pythagore(4, 3, 1, 2, 1000)
b

In [ ]:
#Create a function that compute pair-wise comparisons between each harmonic of one peak with all harmonics of other peaks
#and returns matching peaks and their harmonic position
bounds = 0.5
for p in range(len(multi_harmonics)):
    for h in range(len(multi_harmonics[0])):
        if p != 
        
        

In [188]:
a, b, c, d= compute_pythagore(3, 2, 1, 3)
a

{'unison': 1.0,
 'minor second': 3.5555555555555545,
 'major second': 0.75,
 'minor third': 2.666666666666666,
 'major third': 0.5625,
 'perfect fourth': 2.0,
 'diminished fifth': 7.111111111111108,
 'augmented fourth': 0.4218749999999999,
 'perfect fifth': 1.5,
 'minor sixth': 5.333333333333332,
 'major sixth': 1.125,
 'minor seventh': 4.0,
 'major seventh': 0.84375,
 'octave': 3}

In [14]:
10/9

1.1111111111111112

In [118]:
h, i = harmonic_tuning(20, 2)
h


[1.0, 2.0, 1.75, 1.5, 1.25, 1.125, 1.375, 1.625, 1.875, 1.0625, 1.1875]

In [ ]:
a, b, ratio_mult = oct_subdiv(1.5, 0.02, 2, 1)
print(ratio_mult)


pyth_dict, pyth_dict_list = compute_pythagore(3, 2, 1, 2)
print(pyth_dict_list)

In [ ]:
## Calculate hertz to cents

c = 1200 × log2 (f2 / f1)

# Dissonance

In [158]:
#Functions taken from : https://gist.github.com/endolith/118429

def euler(*numbers):
    """Euler's "gradus suavitatis" (degree of sweetness) function
    
    Return the "degree of sweetness" of a musical interval or chord expressed 
    as a ratio of frequencies a:b:c, according to Euler's formula
    
    Greater values indicate more dissonance
        
    """
    factors = prime_factors(lcm(*reduced_form(*numbers)))
    return 1 + sum(p - 1 for p in factors)


def wiseman(a, b):
    """Return a value corresponding to the dissonance of a musical interval 
    of ratio a:b
    
    This is taken from Gus Wiseman's paper at 
    http://www.nafindix.com/math/sensory.pdf
    
    It apparently is derived from the total period of two tones, so a tone 
    of period 2 and a tone of period 3 when summed will give a total period 
    of 6 (which is LCM(2,3))
    
    Since a ratio of frequencies is just the inverse of a ratio of periods, 
    and inverting the terms has no effect on this formula, I *think* it's 
    valid to just plug in frequency ratios, too.
    
    Same as "harmonic value"?
    TODO: a/gcd(all) and b/gcd(all) is just the reduced form, right?  and if a,b are in reduced form, then lcm(a,b) =a*b, right?  So this is just a*b when a,b are in reduced form.  So this should be benedetti().  https://en.wikipedia.org/wiki/Giambattista_Benedetti#Music
    product(*reduced_form(a,b))
    
    http://xenharmonic.wikispaces.com/Benedetti+height
    
    """
    return lcm(a / gcd(a, b), b / gcd(a, b));


def tenney(a, b):
    """Return Tenney's harmonic distance of a musical interval of ratio a:b
    
    "Tenney's HD function, in one of its simplest forms, is perhaps the most 
    direct. It is HD(a/b) = log(ab)"
    
    Also called "Tenney height" with log2: http://xenharmonic.wikispaces.com/Tenney+Height
    Tenney's paper says "harmonic distance" and "log", but relates it to a lattice so maybe the log2 is implied?
    
    http://lumma.org/tuning/faq/#heights says
    Of these, the product of numerator and denominator, often
    written n*d, for a ratio n/d in lowest terms, has been found to
    best agree with informal rankings by listeners as well as
    published psychoacoustic data.  
    
    The n*d rule is also called Tenney height, after James Tenney^1,
    
    Tenney height generalizes to geomean(a*b*c...) for a chord
    a:b:c..., which gives sqrt(n*d) for dyads.
    
    """
    return log(product(*reduced_form(a,b)));

## Dissonance curve from Sethares

In [161]:
from dissonant import harmonic_tone, dissonance, pitch_to_freq
freqs, amps = harmonic_tone(pitch_to_freq([0, 2.5, 7, 12]), n_partials=10)
d = dissonance(freqs, amps, model='sethares1993')
d

1.022126263286898

# PyTuning

In [151]:
import pytuning as pt
import sympy as sp
from pytuning.scales import create_edo_scale
edo_12_scale = create_edo_scale(53)
edo_12_scale

x = pt.find_best_modes(peak_ratios_rebound, 7)
x = pt.find_best_modes(harm_ratios[0], 7)
x
#major_mask = (0,2,4,5,7,9,11,12)
#major_mode = mask_scale(edo_12_scale, major_mask)
#major_mode

NameError: name 'harm_ratios' is not defined

In [90]:
#This function is similar to the harmonic_tuning function built in BioTuning Package

from pytuning.scales import create_harmonic_scale
scale = create_harmonic_scale(1,20)
scale

h, i = harmonic_tuning(12, 2)
h



[1.0, 2.0, 1.75, 1.5, 1.25, 1.125, 1.375]

In [ ]:
#Computes ratios of the harmonic scale

harm_ratios = compute_peak_ratios(h)
harm_ratios[0]

In [ ]:
h = EEG_harmonics_mult(peaks, 12, 0)
h

In [99]:
z = pt.scales.create_equal_interval_scale(3/2, scale_size=12)
z

[1.0,
 1.05349794238683,
 1.12500000000000,
 1.18518518518518,
 1.26562500000000,
 1.33333333333333,
 1.40466392318244,
 1.5,
 1.58024691358025,
 1.68750000000000,
 1.77777777777778,
 1.89843750000000,
 2]

In [111]:
sorted(ratios_tet)

[1.0136432647705078,
 1.06787109375,
 1.125,
 1.20135498046875,
 1.265625,
 1.3515243530273438,
 1.423828125,
 1.5,
 1.601806640625,
 1.6875,
 1.802032470703125,
 1.8984375]

In [406]:
intervals     = [22, 5, 9, 15]
multiplicities = [1,1,1,1]
scale         = pt.scales.create_euler_fokker_scale(intervals, multiplicities)
scale

[2, 8, 11, 18, 29, 33]

In [162]:
3/2**(1/9)

2.7776241368618715

## PyAudio

In [ ]:
def notes2play(array, fund):
    scale2play = []
    scale2play = [fund*x for x in array]
    
    return scale2play
    

In [ ]:
import pyaudio
import struct
import math

FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100

p = pyaudio.PyAudio()


def data_for_freq(frequency: float, time: float = None):
    """get frames for a fixed frequency for a specified time or
    number of frames, if frame_count is specified, the specified
    time is ignored"""
    frame_count = int(RATE * time)

    remainder_frames = frame_count % RATE
    wavedata = []

    for i in range(frame_count):
        a = RATE / frequency  # number of frames per wave
        b = i / a
        # explanation for b
        # considering one wave, what part of the wave should this be
        # if we graph the sine wave in a
        # displacement vs i graph for the particle
        # where 0 is the beginning of the sine wave and
        # 1 the end of the sine wave
        # which part is "i" is denoted by b
        # for clarity you might use
        # though this is redundant since math.sin is a looping function
        # b = b - int(b)

        c = b * (2 * math.pi)
        # explanation for c
        # now we map b to between 0 and 2*math.PI
        # since 0 - 2*PI, 2*PI - 4*PI, ...
        # are the repeating domains of the sin wave (so the decimal values will
        # also be mapped accordingly,
        # and the integral values will be multiplied
        # by 2*PI and since sin(n*2*PI) is zero where n is an integer)
        d = math.sin(c) * 32767
        e = int(d)
        wavedata.append(e)

    for i in range(remainder_frames):
        wavedata.append(0)

    number_of_bytes = str(len(wavedata))  
    wavedata = struct.pack(number_of_bytes + 'h', *wavedata)

    return wavedata


def play(frequency: float, time: float):
    """
    play a frequency for a fixed time!
    """
    frames = data_for_freq(frequency, time)
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True)
    stream.write(frames)
    stream.stop_stream()
    stream.close()


#if __name__ == "__main__":
    
    
        


In [ ]:
scale2play = notes2play(d, 100)
scale2play
for i in scale2play:
    play(i, 1)
    